<img align="center" src="https://raw.githubusercontent.com/FUlyankin/Parsers/master/images%20/cats.jpg" height="1200" width="1200"> 

# <center> ML для маркетинга. <br>  <br> Cюжетная арка 3: сегментация </center>

В этом семестре мы с вами пройдём через несколько сюжетных арок. Третьей и заключительной такой аркой будет сегментация. 


#  Глава II (Вконтакте и его мальчик) 

Ознакомиться со способами взаимодействия с Вконтакте можно по [ссылке](https://vk.com/dev/manuals) на отличную документацию. Для того, чтобы получить доступ к выгрузке данных, нужно пройти ряд бюрократических процедур (о, боже, эти два предложения были так бюрократически сформулированы, что мне захотелось отстоять в очереди).

Первая такая процедура заключается в создании своего приложения. Для этого переходим по [ссылке](http://vk.com/editapp?act=create) и проходимся по необходимым шагам:

<img align="center" src="pictures/app_creation_1.png" height="600" width="600">

После подтверждения своей личности по номеру телефона, попадаем на страницу свежесозданного приложения
<img align="center" src="pictures/app_creation_2.png" height="600" width="600">

Слева нам будем доступна вкладка с настройками, перейдя в неё мы увидим все необходимые нам для работы с приложением параметры:
<img align="center" src="pictures/app_creation_3.png" height="600" width="600">


Для работы с частью методов API этого вполне достаточно (обычно в заголовке такого метода стоит соответствующая пометка). Для части методов, используемых нами, может понадобиться ключ доступа.

Для того, чтобы получить его, необходимо сделать ещё пару странных манипуляций:

Переходим по ссылке вида (на месте звездочек должен стоять ID созданного вами приложения):

> https://oauth.vk.com/authorize?client_id=**********&scope=14&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.16&response_type=token

<img align="center" src="pictures/app_creation_4.png" height="600" width="600">

В итоге по этому запросу будет сформирована ссылка следующего вида:
> https://oauth.vk.com/blank.html#access_token=25b636116ef40e0718fe4d9f382544fc28&expires_in=86400&user_id=*******

Первый набор знаков - `access token`, т.е. маркер доступа. Вторая цифра (`expires_in=`) время работы маркера доступа в секундах (одни сутки). По истечению суток нужно будет получить новый маркер доступа. Последняя цифра (`user_id=`) ваш ID Вконтакте. Нам в дальнейшем понадобится маркер доступа. Для удобства сохраним его в отдельном файле или экспортируем в глобальную область видимости. В целях безопасности ваших данных не стоит нигде светить токенами и тем более выкладывать их в открытый доступ.

Обратите внимание на ссылку, по которой мы делали запрос на предоставление токена. Внутри неё находится странный параметр `scope=14.` Эта загадочная цифра есть ничто иное, как права доступа к социальной сети. Подробнее познакомиться с взаимно-однозначным соответствием между числами и правами можно в документации. Например, если мы хотим получить доступ к друзьям, фото и аудио, мы подставим в scope цифру 2+4+8=14.

> __ВАЖНО:__ Если отдать свой токен другому человеку, у него будет доступ к вашему аккаунту. Он легко может выкачать ваши личные переписки и даже удалить страницу. Не светите свой токен в интернетах и берегите его с молоду. 

# 1. Впервые врываемся в вк и узнаём как нас зовут

In [1]:
import requests # библиотека для походов в интернет

Подгрузим токен.

In [4]:
# мой номер странички
myid = '6045249'# вставить номер странички

# версия используемого API
version = '5.73' 

# подгружаем токен
with open('secret_token.txt') as f:
    token = f.read()

Чтобы скачать что-то из контакта, надо сделать ссылку и сходить по ней пакетом `requests`. Ссылка должна будет включать в себя метод (что мы просим у вк) и параметры (насколько много). Мы будем просто заменять эти две штуки и выкачивать разные вещи. 

In [5]:
method = 'users.get'
parameters = 'user_ids=' + myid

url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + str(token) + '&v=' + version

response = requests.get(url) 
response.json()

{'response': [{'id': 6045249,
   'first_name': 'Филипп',
   'last_name': 'Ульянкин'}]}

Если вы всё сделали правильно, то у вас выводится имя и фамилия.  Залезем в [документацию](http://vk.com/editapp?act=create) и скачаем что-нибудь по ней. Например, всех людей из группы: https://vk.com/hsemem

Если открыть первый пост в группе (нажать на дату его публикации), появиться ссылка https://vk.com/hsemem?w=wall-139105204_48261

Первые числа это и есть id.

In [6]:
mem_id = '139105204'
offset = '1000'

method = 'groups.getMembers' # хотим скачать участников группы

              # в документации видим что первый параметр метода это id группы
              # второй параметр это сдвиг (максимум 1000 человек)
parameters = 'group_id=' + mem_id + '&offset=' + offset
parameters

'group_id=139105204&offset=1000'

Снова подставим метод и параметры в ссылку. 

In [7]:
url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + str(token) + '&v=' + version

Идём по ней. 

In [8]:
response = requests.get(url) 
infa = response.json()

Всё скачалось. 

In [9]:
# infa

Давайте завернем это в функцию. 

In [10]:
def vk_download(method, parameters):
    
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + str(token) + '&v=' + version
    response = requests.get(url) 
    infa = response.json()

    return infa 

Дальше будем всегда использовать только её. Например, чтобы скачать информацию о пользователях, вот так (fields мы нашли в документации, оказалось что можно уточнить что мы хотим скачать про пользователя). 

In [11]:
infa = vk_download('users.get','user_ids=6045249,494821147&fields=bdate,city')
infa

{'response': [{'id': 6045249,
   'first_name': 'Филипп',
   'last_name': 'Ульянкин',
   'bdate': '3.10.1994',
   'city': {'id': 1, 'title': 'Москва'}},
  {'id': 494821147,
   'first_name': 'Alina',
   'last_name': 'Kobets',
   'bdate': '18.2.1999',
   'city': {'id': 194, 'title': 'Ташкент'}}]}

Завернём это в табличку.

In [12]:
import pandas as pd 
df = pd.DataFrame(infa['response'])
df

,bdate,city,first_name,id,last_name
0,3.10.1994,"{'id': 1, 'title': 'Москва'}",Филипп,6045249,Ульянкин
1,18.2.1999,"{'id': 194, 'title': 'Ташкент'}",Alina,494821147,Kobets


Хочется распаковать словарик с городами, мы уже умеем это делать!

In [13]:
df['real_city'] = df['city'].apply(lambda w: w['title'])
df

,bdate,city,first_name,id,last_name,real_city
0,3.10.1994,"{'id': 1, 'title': 'Москва'}",Филипп,6045249,Ульянкин,Москва
1,18.2.1999,"{'id': 194, 'title': 'Ташкент'}",Alina,494821147,Kobets,Ташкент


Что-то мы отвлеклись. Давайте вернёмся к основной цели и скачаем всех подписчиков паблика с помощью нашей функции. 

In [14]:
offset = 0

cur_infa = vk_download('groups.getMembers', 
            'group_id=139105204&offset=' + str(offset))

len(cur_infa['response']['items']) # скачали первую тысячу

1000

Надо скачать всех! Напишем цикл.

In [15]:
list(range(0, 10000, 1000)) # какую тысячу качать 

[0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]

In [16]:
import time

infa = [ ] # сюда буду сохранять людей

for offset in range(0, 49000, 1000):
    
    time.sleep(0.4) # вк согласен работать 3 раза в секунду, 
                    # эта строка нужна, чтобы он не ругался 
    
    cur_infa = vk_download('groups.getMembers', 
                'group_id=139105204&offset=' + str(offset))

    infa.extend(cur_infa['response']['items'])

In [17]:
len(infa)

49000

Вот пятидесятый человек. 

In [18]:
infa[-50]

523421775

Скачаем его стену! Найдём в документации метод и параметры.

In [21]:
wall = vk_download('wall.get', 'owner_id='+str(infa[50])+'&count=100')
wall = wall['response']['items']
len(wall)

100

In [22]:
wall[-1]['text']

'Криотерапия. Бесплатно. Везде @ Vorkuta Airport'

In [23]:
# все записи со стены
[cur['text'] for cur in wall]

['Скоро сессия',
 'Все главные символы города - в одной фотографии',
 'Когда ты препод - ты можешь проводить любые творческие конкурсы (даже сам об этом не догадываясь). Лайк э босс. Спасибо 1 курсу! А конкретно [id185820259|Катя Гревцева], [id246192678|Майя Сидякова], [id143110287|Елизавета Малыхина], [id55552388|Анастасия Докукина], [id262290182|Лина Балашова], [id170697938|Юля Королева]',
 'В бары всегда ходить приятно, а 1 сентября - еще и умно',
 'Если долго смотреть на воду - можно заметить в ней меня )\nСпасибо за это [id64639098|Нураше]',
 'Прыгать рыбкой было страшно, я струсил и прыгал солдатиком',
 'Кто кого: розы или колючая проволока?',
 'Вечерние променады',
 'ЛПШ закончилась. Да здравствует следующая ЛПШ!\n#лучшийоргком #лпш #лпш_мгу #lpshine',
 '33 - это когда тебе все еще приходят sms-поздравления в рассылке от кальянных и ночных клубов, но уже больше вчитываешься в акции и скидки в поздравлениях от аптек. И знаете что? Между возрастом Христа и Ильи Муромца я выбираю в

Со стены можно ещё скачать текст репостов.

In [24]:
wall[-1]['copy_history'][0]['text']

KeyError: 'copy_history'

In [25]:
# репосты есть не везде
'copy_history' in wall[-1]

False

In [26]:
# репосты есть не везде
'copy_history' in wall[0]

False

Будем проверять их наличие и брать текст, если он есть. 

In [27]:
[cur['copy_history'][0]['text'] if 'copy_history' in cur else '' for cur in wall]

['',
 '',
 '',
 'Когда вы листаете вашу ленту в соцсетях, вы, скорее всего, встречаете статьи про «10 секретов управления поколением Y» или «Кто такие миллениалы и как их мотивировать». Мы читаем их, удивляемся, но не знаем, что за этим кроется — вольная фантазия автора или правда. Неужели те, кому сейчас 20–30 лет, сильно отличаются от предыдущих поколений? \n\nНа лекции психолога Александра Рикеля вы узнаете, действительно ли поколение Y заслуженно называют вечными детьми, насколько они рациональны, как выстраивают карьеру, а как — романтические отношения.\n\nЗарегистрироваться на эту и другие лекции: https://vk.cc/9IijVd',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Тренировочная база юных исследователей\nоткрывает свои двери\n28 ФЕВРАЛЯ 18:30 АУДИТОРИЯ УТОЧНЯЕТСЯ\n\nПервый год на факультете?\nИли второй, но такое ощущение, что еще первый?\n\nПоследний четверг каждого месяца мы с вами будем отрабатывать все необходимые навыки для проведения и написания научных рабо

Чтобы скачать все записи всех пользователей, надо снова написать цикл. :) 